In [1]:
import requests
from api_key import GAODE_WEATHER_API_KEY

from crewai.tools import tool
@tool
def get_weather(city_name:str)->dict:
    '''根据输入的城市名，使用高德接口查询当地天气'''
    # 构建请求URL
    url = f'https://restapi.amap.com/v3/weather/weatherInfo?key={GAODE_WEATHER_API_KEY}&city={city_name}&extensions=all'
    print(url)
    try:
        # 发送GET请求
        response = requests.get(url)
        # 检查请求是否成功
        if response.status_code == 200:
            # 解析响应内容
            data = response.json()
            # 检查API响应状态
            if data.get('status') == '1' and data.get('infocode') == '10000':
                return data  # 返回天气数据
            else:
                return {}  # 返回空结果，因为API返回了错误信息
        else:
            return {}  # 返回空结果，因为HTTP状态码不是200
    except Exception as e:
        print(f"An error occurred: {e}")
        return {}  # 返回空结果，因为请求过程中发生了异常


In [2]:
from api_key import SERPER_API_KEY,TAVILY_API_KEY
import os
os.environ["SERPER_API_KEY"] = SERPER_API_KEY  # serper.dev API key

import litellm
litellm.suppress_debug_info=True

# 设置ollama模型
# os.environ["OPENAI_API_BASE"] = 'http://192.168.3.155:11434' 
# os.environ["OPENAI_MODEL_NAME"] ='ollama_chat/qwen2.5:latest'
# os.environ["OPENAI_API_KEY"] ='EMPTY'

from crewai.llm import LLM
llm=LLM(model="ollama_chat/qwen2.5:latest", base_url="http://192.168.3.155:11434")
embedder_config={
        "provider": "ollama",
        "config": {
            "model": 'quentinz/bge-large-zh-v1.5:latest',
            "base_url": 'http://192.168.3.155:11434'
        }
    }

from crewai_tools import SerperDevTool
search_tool = SerperDevTool(n_results=5)
tools=[search_tool,get_weather]

# from crewai_tools import LlamaIndexTool
# from llama_index.tools.tavily_research import TavilyToolSpec 
# tavily_research_tool=TavilyToolSpec(api_key=TAVILY_API_KEY)
# research_tools=tavily_research_tool.to_tool_list()
# tools = [LlamaIndexTool.from_tool(t) for t in research_tools]

# 创建一个带有内存和冗长模式的高级研究员代理人
from crewai import Agent
manager = Agent(
    role='旅行社经理',
    goal='针对 {destination} 设计旅游计划',
    verbose=True,
    memory=True,
    backstory=(
        "专业的旅行社经理，根据用户的需求，设计客户满意的旅行计划"
    ),
    llm=llm,
    embedder_config=embedder_config,
    tools=tools,
    allow_delegation=True,
    max_rpm=5,
    max_execution_time=15,
)

# 创建一个作家代理人，具有自定义工具和委派能力
writer = Agent(
    role='百事通',
    goal='输出 {destination} 的游玩攻略',
    verbose=True,
    memory=True,
    backstory=(
        "长期在{destination}居住，对当地历史及人文景观十分了解，经常编写当地的游玩攻略"
    ),
    llm=llm,
    embedder_config=embedder_config,
    tools=tools,
    allow_delegation=False,
    max_rpm=5,
    max_execution_time=30,
)

In [3]:
from crewai import Task

# 研究任务
research_task = Task(
  description=(
    "编写关于 {destination} 的游玩攻略，比如历史、人文、饮食等"
    "你必须从本地人的角度去考虑"
  ),
  expected_output='整理输出{destination} 游玩推荐项目及原因',
  tools=tools,
  agent=writer,
)

# 具有语言模型配置的写作任务
write_task = Task(
  description=(
    "根据旅游天数：{day}、预算：{budget}，撰写一篇关于 {destination} 的旅游攻略"
    "如果提供了旅游倾向：{tendency}，必须则优先满足，并且根据天气情况安排旅游项目"
    "文章首先输出旅游时间内的天气情况，然后再输出攻略"
    "文章通俗易懂，平易近人，可以适当加一个emoj表情"
  ),
  expected_output='输出定制化的旅游攻略，格式为 markdown。',
  tools=tools,
  agent=manager,
  async_execution=False,
  output_file='{destination}-travel-plan.md'  # 输出定制的示例
)

In [4]:
from crewai import Crew, Process

# 以一些增强配置形成以技术为中心的团队
crew = Crew(
    agents=[writer,manager],
    tasks=[research_task,write_task],
    process=Process.sequential,  # 可选：顺序任务执行是默认设置
    memory=False,
    cache=True,
    verbose=True,
    manager_llm=[llm],
    embedder=embedder_config
)

In [5]:
# 以增强反馈开始任务执行流程
travel_require={
    'destination': '广州',
    'day': 3,
    'budget': 2000,
    'tendency':'无'
}


result = crew.kickoff(inputs=travel_require)
print(result)

# Agent: 百事通
## Task: 编写关于 广州 的游玩攻略，比如历史、人文、饮食等你必须从本地人的角度去考虑


# Agent: 百事通
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"\\u5e7f\\u5dde \\u5386\\u53f2 \\u4eba\\u6587 \\u996e\\u98df \\u70ed\\u95e8\\u666f\\u70b9\"}"
## Tool Output: 

Search results: Title: 广州市历史景点 - Tripadvisor
Link: https://cn.tripadvisor.com/Attractions-g298555-Activities-c47-t17-Guangzhou_Guangdong.html
Snippet: 推荐观光体验(360) · 3. 怀圣寺 · 4. 宝墨园 · 5. 清真先贤古墓 · 6. 番禺沙湾古镇 · 7. 小洲村 · 8. 永庆坊 · 9. 南越王宫博物馆 · 10. 黄埔军校旧址纪念馆.
---
Title: 广州市10 大历史与古迹游览 - Tripadvisor
Link: https://cn.tripadvisor.com/Attractions-g298555-Activities-c42-t228-Guangzhou_Guangdong.html
Snippet: 广州热门景点有白云山、中山纪念堂、六榕寺、荔枝湾风光、沙面岛、花城广场、广州塔、上下九步行街、云台园、陈家祠、越秀公园、南越王陵博物馆等。
---
Title: 广州最著名的六大“人文景点”，适合带父母一日游，你一定用得上 - 搜狐
Link: https://www.sohu.com/a/329438925_120178803
Snippet: 沙湾古镇，获评国家4A级旅游景区，每年都吸引来自世界各地的游客。这里古镇始建于宋代，因地处古海湾半月型的沙滩之畔，故名“沙湾”。 沙 ...
---
Title: “文旅最广州系列名录”之十大景区发布令人心向往之的广州这里风景独好
Link: http://wglj.gz.gov.cn/ggfw/lyl/content/post

In [6]:
# 以增强反馈开始任务执行流程
travel_require={
    'destination': '广州',
    'day': 3,
    'budget': 2000,
    'tendency':'第三天上午我想去大夫山玩'
}


result = crew.kickoff(inputs=travel_require)
print(result)

# Agent: 百事通
## Task: 编写关于 广州 的游玩攻略，比如历史、人文、饮食等你必须从本地人的角度去考虑


# Agent: 百事通
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"\\u5e7f\\u5dde \\u5386\\u53f2 \\u4eba\\u6587 \\u996e\\u98df\"}"
## Tool Output: 

Search results: Title: “食在广州”背后的文化交流 - 南方网
Link: https://news.southcn.com/node_54a44f01a2/020f12acff.shtml
Snippet: “食在广州”两千年前始于南越王宴​ 在南越王博物馆展示出土的文物中，从“御膳珍馐”到炊具容器，一应俱全。 两千多年前的南越王，已经将丰富食材烹制成了佳 ...
---
Title: 追忆粤菜两千年历史，探寻“食在广州”文化底蕴
Link: http://www.qb.gd.gov.cn/ztzl/2021ycsf/xwdt/content/post_1028903.html
Snippet: 粤菜的起源可追溯两千多年前，南越王赵佗治理岭南时，当地经济和文化有所发展，饮食烹饪演化成型。 汉文帝时，南越王赵佗归附汉朝，进一步促成了中原文化与岭南 ...
---
Title: 粤菜之美在于和与合 - 广州市人民政府
Link: https://www.gz.gov.cn/zlgz/tsgz/content/post_8033813.html
Snippet: 据中山大学周松芳教授考证，虽然典籍中记载的广州饮食有逾两千年的历史，但“食在广州”之驰名始于晚清。 广州“一口通商”时代，世界各地商贾聚集十三行。 ...
---
Title: 独具特色的广州饮食文化 - 新浪新闻
Link: https://news.sina.com.cn/o/2009-12-29/180416851193s.shtml
Snippet: 广州主要以粤菜为主，粤菜的形成历史悠久。早在汉魏时期，一批批中原人移居岭南，使广州饮食文化深受中原饮食文化的影响。鸦片战争后，欧美各国的传教 ...
---
Title: 现代作家